In [1]:
import os
import numpy as np
import nibabel as nib

# CT 파일이 있는 폴더 경로
data_root = "/SSD5_8TB/Daniel/09_registformer_hugging_face/mem_eff_registformer/data/synthrad2023_mr-ct_pelvis"

# 환자 리스트
patients = sorted(os.listdir(data_root))
patients = [p for p in patients if os.path.isdir(os.path.join(data_root, p))]

# CT max 값들 저장용 리스트
ct_max_values = []

for patient in patients:
    ct_path = os.path.join(data_root, patient, "ct.nii.gz")
    if not os.path.exists(ct_path):
        continue

    ct_data = nib.load(ct_path).get_fdata()
    ct_max = ct_data.max()
    ct_max_values.append(ct_max)

# 모든 CT max값들 중 최소값 출력
if ct_max_values:
    min_of_max = min(ct_max_values)
    print(f"CT max값들 중 최소값: {min_of_max:.3f}")
else:
    print("CT 파일이 없습니다.")


CT max값들 중 최소값: 1389.000


In [ ]:
ㅇㅇ

In [14]:
# Unified inter-slice consistency script (local-only)
# - TV/nTV, Edge-TV, Lap-TV, in-plane sharpness, Otsu mask, spacing_z
# - Summary & per-slice CSVs + plots
# - Keeps original absolute paths and filenames

import os, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- I/O backends ----
nib = None
sitk = None
try:
    import nibabel as nib
except Exception:
    nib = None
if nib is None:
    try:
        import SimpleITK as sitk
    except Exception:
        sitk = None

# ---- Optional filters ----
has_scipy = False
try:
    from scipy.ndimage import sobel, laplace
    has_scipy = True
except Exception:
    has_scipy = False

# ---- Paths (KEEP) ----
paths = {
    "ProposedSynth_Epoch87_fullimage": r"/SSD5_8TB/Daniel/JBHI 저널/Output(result)/MR-CT/Registformer_MrCtPelvis_ver2_ProposedSynthesis_NoRandTranslation_VoxelmorphProposedSynth_Test_Epoch87_fullimage/preds_a_1PA004.nii.gz",
    "MUNIT_Lcx_rotate": r"/SSD5_8TB/Daniel/JBHI 저널/Output(result)/MR-CT/I2I_translation/MUNIT+Lcx(예전)_rotate/preds_b_1PA004.nii.gz",
    "VoxelMorph2D_warped": r"/SSD5_8TB/Daniel/JBHI 저널/Output(result)/MR-CT/Registration/voxelmorph2D/warped_img_1PA004.nii.gz",
}
# Base output dir (KEEP)
base_out_dir = r"/SSD5_8TB/Daniel/JBHI 저널/Slice간의 일관성 정량값"
plots_dir = os.path.join(base_out_dir, "jbhi_metrics")
os.makedirs(base_out_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

# ---- Helpers ----
def load_nifti(path):
    if nib is not None:
        img = nib.load(path)
        return img.get_fdata(dtype=np.float32)  # (X,Y,Z)
    elif sitk is not None:
        img = sitk.ReadImage(path)
        arr = sitk.GetArrayFromImage(img)  # (Z,Y,X)
        return np.transpose(arr, (2,1,0)).astype(np.float32)
    else:
        raise RuntimeError("nibabel 또는 SimpleITK가 필요합니다.")

def robust_minmax_norm(vol, p1=1.0, p99=99.0, eps=1e-6):
    lo = np.percentile(vol, p1)
    hi = np.percentile(vol, p99)
    vol = np.clip(vol, lo, hi)
    return (vol - lo) / max(hi - lo, eps)

def otsu_threshold(arr, bins=256):
    arr = arr[np.isfinite(arr)]
    if arr.size == 0:
        return 0.0
    hist, bin_edges = np.histogram(arr, bins=bins, range=(arr.min(), arr.max()))
    w1 = np.cumsum(hist)
    w2 = np.cumsum(hist[::-1])[::-1]
    mu1 = np.cumsum(hist * (bin_edges[:-1] + bin_edges[1:]) / 2) / np.maximum(w1, 1e-12)
    mu2 = (np.cumsum((hist * (bin_edges[:-1] + bin_edges[1:]) / 2)[::-1]) / np.maximum(w2[::-1], 1e-12))[::-1]
    var12 = w1[:-1] * w2[1:] * (mu1[:-1] - mu2[1:])**2
    idx = np.argmax(var12)
    return float((bin_edges[idx] + bin_edges[idx+1]) / 2)

# ---- Metrics ----
def gradient_magnitude(s2d):
    if has_scipy:
        gx = sobel(s2d, axis=0, mode='reflect')
        gy = sobel(s2d, axis=1, mode='reflect')
    else:
        gx = np.zeros_like(s2d); gy = np.zeros_like(s2d)
        gx[1:-1,:] = (s2d[2:,:] - s2d[:-2,:]) / 2.0
        gy[:,1:-1] = (s2d[:,2:] - s2d[:,:-2]) / 2.0
    return np.sqrt(gx**2 + gy**2)

def laplacian2d(s2d):
    if has_scipy:
        return laplace(s2d)
    center = 4*s2d
    n = np.zeros_like(s2d)
    n[1: ,:] += s2d[:-1,:]; n[:-1,:] += s2d[1: ,:]
    n[: ,1:] += s2d[:, :-1]; n[: ,:-1] += s2d[:, 1: ]
    return n - center

def mse(a,b): return float(np.mean((a-b)**2))
def psnr_from_mse(m, data_range=1.0):
    if m <= 1e-12: return float('inf')
    return 20.0*math.log10(data_range) - 10.0*math.log10(m)

def global_ssim(a,b,data_range=1.0,K1=0.01,K2=0.03):
    a = a.astype(np.float64); b = b.astype(np.float64)
    C1 = (K1*data_range)**2; C2 = (K2*data_range)**2
    mu_a = float(np.mean(a)); mu_b = float(np.mean(b))
    var_a = float(np.var(a)); var_b = float(np.var(b))
    cov = float(np.mean((a - mu_a) * (b - mu_b)))
    num = (2*mu_a*mu_b + C1) * (2*cov + C2)
    den = (mu_a**2 + mu_b**2 + C1) * (var_a + var_b + C2)
    if den == 0: return 1.0 if num == 0 else 0.0
    return float(num/den)

def mutual_information(a,b,bins=64):
    a = np.clip(a.ravel(),0,1); b = np.clip(b.ravel(),0,1)
    h, _, _ = np.histogram2d(a,b,bins=bins,range=[[0,1],[0,1]])
    p = h/np.sum(h) + 1e-12
    px = np.sum(p,axis=1,keepdims=True); py = np.sum(p,axis=0,keepdims=True)
    return float(np.sum(p*(np.log(p) - np.log(px) - np.log(py))))

def ncc(a,b):
    a = a.astype(np.float64); b = b.astype(np.float64)
    am, bm = np.mean(a), np.mean(b)
    asd, bsd = np.std(a), np.std(b)
    if asd < 1e-8 or bsd < 1e-8: return 0.0
    return float(np.mean((a-am)*(b-bm)) / (asd*bsd))

def tv_z(vol, mask=None):
    diffs = np.abs(np.diff(vol, axis=2))
    if mask is not None:
        diffs = diffs * mask[:,:,1:]
        denom = np.sum(mask[:,:,1:])
        return float(np.sum(diffs) / max(denom, 1e-12))
    return float(np.mean(diffs))

def tv_z_per_interval(vol, mask=None):
    diffs = np.abs(np.diff(vol, axis=2))
    if mask is not None:
        diffs = diffs * mask[:,:,1:]
        denom = np.sum(mask[:,:,1:], axis=(0,1))
        denom = np.maximum(denom, 1e-12)
        return np.sum(diffs, axis=(0,1)) / denom
    return np.mean(diffs, axis=(0,1))

def tv_xy(vol, mask=None):
    dx = np.abs(np.diff(vol, axis=0))
    dy = np.abs(np.diff(vol, axis=1))
    if mask is not None:
        mx = mask[1:,:,:]; my = mask[:,1:,:]
        dx = dx * mx; dy = dy * my
        denom = np.sum(mx) + np.sum(my)
        return float((np.sum(dx)+np.sum(dy)) / max(denom, 1e-12))
    return float(np.mean(dx) + np.mean(dy))

def edge_tv_z(vol, mask=None):
    Z = vol.shape[2]
    gm = np.zeros_like(vol, dtype=np.float32)
    for z in range(Z):
        gm[:,:,z] = gradient_magnitude(vol[:,:,z])
    return tv_z(gm, mask)

def lap_tv_z(vol, mask=None):
    Z = vol.shape[2]
    lv = np.zeros_like(vol, dtype=np.float32)
    for z in range(Z):
        lv[:,:,z] = laplacian2d(vol[:,:,z])
    return tv_z(lv, mask)

def inplane_sharpness_metrics(vol, mask=None):
    Z = vol.shape[2]
    lap_abs, lap_var, edge_m = [], [], []
    for z in range(Z):
        sl = vol[:,:,z]
        if mask is not None:
            m = mask[:,:,z] > 0
            if not np.any(m):
                lap_abs.append(0.0); lap_var.append(0.0); edge_m.append(0.0); continue
            lap = laplacian2d(sl)[m]; gm = gradient_magnitude(sl)[m]
        else:
            lap = laplacian2d(sl).ravel(); gm = gradient_magnitude(sl).ravel()
        lap_abs.append(float(np.mean(np.abs(lap))))
        lap_var.append(float(np.var(lap)))
        edge_m.append(float(np.mean(gm)))
    return {
        "LaplacianAbs_mean": float(np.mean(lap_abs)),
        "LaplacianVar_mean": float(np.mean(lap_var)),
        "EdgeMag_mean": float(np.mean(edge_m)),
    }

def per_slice_pair_metrics(vol, mask=None):
    Z = vol.shape[2]
    d = {"MSE":[], "PSNR":[], "SSIM_global":[], "MI":[], "NCC":[], "TV":[]}
    intervals = tv_z_per_interval(vol, mask)
    for z in range(Z-1):
        a = vol[:,:,z]; b = vol[:,:,z+1]
        if mask is not None:
            m = (mask[:,:,z] * mask[:,:,z+1]) > 0
            a_sel = a[m] if np.any(m) else a
            b_sel = b[m] if np.any(m) else b
        else:
            a_sel = a; b_sel = b
        mv = mse(a_sel, b_sel)
        d["MSE"].append(mv)
        d["PSNR"].append(psnr_from_mse(mv))
        d["SSIM_global"].append(global_ssim(a_sel, b_sel))
        d["MI"].append(mutual_information(a_sel, b_sel))
        d["NCC"].append(ncc(a_sel, b_sel))
        d["TV"].append(float(intervals[z]))
    for k in d.keys():
        d[k] = np.array(d[k], dtype=np.float64)
    return d

def summarize_all(vol, name, spacing_z=1.0, use_mask=True):
    vol_n = robust_minmax_norm(vol)
    mask = None
    if use_mask:
        thr = otsu_threshold(vol_n)
        mask = (vol_n > thr).astype(np.float32)

    tvz = tv_z(vol_n, mask)
    tvxy = tv_xy(vol_n, mask)

    nz = vol_n[vol_n > 0]
    iqr = float(np.percentile(nz,75) - np.percentile(nz,25)) if nz.size > 0 else 1.0
    nTV_iqr = tvz / max(iqr, 1e-6)
    nTV_xy  = tvz / max(tvxy, 1e-6)

    edge_tv = edge_tv_z(vol_n, mask)
    lap_tv  = lap_tv_z(vol_n, mask)

    curves = per_slice_pair_metrics(vol_n, mask)
    sharp  = inplane_sharpness_metrics(vol_n, mask)

    return {
        "Name": name,
        "NumSlices": int(vol.shape[2]),
        "TV_z": tvz / max(spacing_z, 1e-6),  # spacing 보정
        "TV_xy": tvxy,
        "nTV_z_by_IQR": nTV_iqr,
        "nTV_z_by_TVxy": nTV_xy,
        "Edge_TV_z": edge_tv,
        "Lap_TV_z": lap_tv,
        "HF_LaplacianAbs_mean": sharp["LaplacianAbs_mean"],
        "HF_LaplacianVar_mean": sharp["LaplacianVar_mean"],
        "HF_EdgeMag_mean": sharp["EdgeMag_mean"],
        "PerSlice_Metrics": curves,
    }

# ---- Runner ----
def run(paths, base_out_dir, plots_dir, spacing_z=1.0, use_mask=True):
    os.makedirs(base_out_dir, exist_ok=True)
    os.makedirs(plots_dir, exist_ok=True)

    summaries = []
    per_slice_csvs = {}

    print(f"[INFO] Output dir: {base_out_dir}")
    for name, p in paths.items():
        if not os.path.exists(p):
            print(f"[WARN] Missing file: {p}")
            continue
        print(f"[LOAD] {name}: {p}")
        vol = load_nifti(p)
        if vol.ndim != 3:
            raise ValueError(f"{name}: expected 3D, got {vol.shape}")

        S = summarize_all(vol, name, spacing_z=spacing_z, use_mask=use_mask)
        summaries.append(S)

        # per-slice CSV (KEEP location/pattern)
        df = pd.DataFrame({k: v for k, v in S["PerSlice_Metrics"].items()})
        per_csv = os.path.join(base_out_dir, f"per_slice_{name}.csv")
        df.to_csv(per_csv, index=False)
        per_slice_csvs[name] = per_csv
        print(f"[SAVE] Per-slice metrics -> {per_csv}")

    if len(summaries) == 0:
        raise RuntimeError("No volumes processed. Check file paths.")

    # Summary CSV (KEEP filename)
    rows = []
    for S in summaries:
        rows.append({
            "Name": S["Name"],
            "NumSlices": S["NumSlices"],
            "TV_z": S["TV_z"],
            "TV_xy": S["TV_xy"],
            "nTV_z_by_IQR": S["nTV_z_by_IQR"],
            "nTV_z_by_TVxy": S["nTV_z_by_TVxy"],
            "Edge_TV_z": S["Edge_TV_z"],
            "Lap_TV_z": S["Lap_TV_z"],
            "HF_LaplacianAbs_mean": S["HF_LaplacianAbs_mean"],
            "HF_LaplacianVar_mean": S["HF_LaplacianVar_mean"],
            "HF_EdgeMag_mean": S["HF_EdgeMag_mean"],
            "PSNR_mean": float(np.mean(S["PerSlice_Metrics"]["PSNR"])),
            "SSIM_global_mean": float(np.mean(S["PerSlice_Metrics"]["SSIM_global"])),
            "MI_mean": float(np.mean(S["PerSlice_Metrics"]["MI"])),
            "NCC_mean": float(np.mean(S["PerSlice_Metrics"]["NCC"])),
            "TV_interval_mean": float(np.mean(S["PerSlice_Metrics"]["TV"])),
        })
    summary_df = pd.DataFrame(rows)
    summary_path = os.path.join(base_out_dir, "inter_slice_consistency_summary.csv")
    summary_df.to_csv(summary_path, index=False)
    print(f"[SAVE] Summary -> {summary_path}")

    # Plots (save into jbhi_metrics subfolder)
    def plot_metric(metric_key, ylabel):
        plt.figure()
        for S in summaries:
            y = S["PerSlice_Metrics"][metric_key]
            x = np.arange(1, len(y)+1)
            plt.plot(x, y, label=S["Name"])
        plt.xlabel("Adjacent slice index (z → z+1)")
        plt.ylabel(ylabel)
        plt.legend()
        out = os.path.join(plots_dir, f"plot_{metric_key}.png")
        plt.savefig(out, dpi=150, bbox_inches="tight")
        plt.close()
        print(f"[PLOT] {metric_key} -> {out}")

    for k, lab in [
        ("SSIM_global", "Global SSIM (z,z+1)"),
        ("PSNR", "PSNR (dB) (z,z+1)"),
        ("MI", "Mutual Information (z,z+1)"),
        ("NCC", "Normalized Cross-Correlation (z,z+1)"),
        ("TV", "Total Variation per z-interval"),
    ]:
        try:
            plot_metric(k, lab)
        except Exception as e:
            print(f"[WARN] Plot fail ({k}): {e}")

    print("\n=== Summary (lower is better for TV metrics; higher is better for PSNR/SSIM/MI/NCC) ===")
    print(summary_df.to_string(index=False))

    return summary_path, per_slice_csvs

# ---- Execute ----
if __name__ == "__main__":
    run(paths, base_out_dir, plots_dir, spacing_z=1.0, use_mask=True)


[INFO] Output dir: /SSD5_8TB/Daniel/JBHI 저널/Slice간의 일관성 정량값
[LOAD] ProposedSynth_Epoch87_fullimage: /SSD5_8TB/Daniel/JBHI 저널/Output(result)/MR-CT/Registformer_MrCtPelvis_ver2_ProposedSynthesis_NoRandTranslation_VoxelmorphProposedSynth_Test_Epoch87_fullimage/preds_a_1PA004.nii.gz
[SAVE] Per-slice metrics -> /SSD5_8TB/Daniel/JBHI 저널/Slice간의 일관성 정량값/per_slice_ProposedSynth_Epoch87_fullimage.csv
[LOAD] MUNIT_Lcx_rotate: /SSD5_8TB/Daniel/JBHI 저널/Output(result)/MR-CT/I2I_translation/MUNIT+Lcx(예전)_rotate/preds_b_1PA004.nii.gz
[SAVE] Per-slice metrics -> /SSD5_8TB/Daniel/JBHI 저널/Slice간의 일관성 정량값/per_slice_MUNIT_Lcx_rotate.csv
[LOAD] VoxelMorph2D_warped: /SSD5_8TB/Daniel/JBHI 저널/Output(result)/MR-CT/Registration/voxelmorph2D/warped_img_1PA004.nii.gz
[SAVE] Per-slice metrics -> /SSD5_8TB/Daniel/JBHI 저널/Slice간의 일관성 정량값/per_slice_VoxelMorph2D_warped.csv
[SAVE] Summary -> /SSD5_8TB/Daniel/JBHI 저널/Slice간의 일관성 정량값/inter_slice_consistency_summary.csv
[PLOT] SSIM_global -> /SSD5_8TB/Daniel/JBHI 저널/Slice

In [ ]:
ㅇㅇ

In [15]:
# Unified inter-slice consistency script (local-only)
# - TV/nTV, Edge-TV, Lap-TV, in-plane sharpness, Otsu mask, spacing_z
# - Summary & per-slice CSVs + plots
# - Keeps original absolute paths and filenames

import os, math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---- I/O backends ----
nib = None
sitk = None
try:
    import nibabel as nib
except Exception:
    nib = None
if nib is None:
    try:
        import SimpleITK as sitk
    except Exception:
        sitk = None

# ---- Optional filters ----
has_scipy = False
try:
    from scipy.ndimage import sobel, laplace
    has_scipy = True
except Exception:
    has_scipy = False

# ---- Paths (KEEP) ----
paths = {
    "ProposedSynth_Epoch87_fullimage": r"/SSD5_8TB/Daniel/JBHI 저널/Output(result)/MR-CT/Registformer_MrCtPelvis_ver2_ProposedSynthesis_NoRandTranslation_VoxelmorphProposedSynth_Test_Epoch87_fullimage/preds_a_1PC066.nii.gz",
    "MUNIT_Lcx_rotate": r"/SSD5_8TB/Daniel/JBHI 저널/Output(result)/MR-CT/I2I_translation/MUNIT+Lcx(예전)_rotate/preds_b_1PC066.nii.gz",
    "VoxelMorph2D_warped": r"/SSD5_8TB/Daniel/JBHI 저널/Output(result)/MR-CT/Registration/voxelmorph2D/warped_img_1PC066.nii.gz",
}
# Base output dir (KEEP)
base_out_dir = r"/SSD5_8TB/Daniel/JBHI 저널/Slice간의 일관성 정량값"
plots_dir = os.path.join(base_out_dir, "jbhi_metrics")
os.makedirs(base_out_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)

# ---- Helpers ----
def load_nifti(path):
    if nib is not None:
        img = nib.load(path)
        return img.get_fdata(dtype=np.float32)  # (X,Y,Z)
    elif sitk is not None:
        img = sitk.ReadImage(path)
        arr = sitk.GetArrayFromImage(img)  # (Z,Y,X)
        return np.transpose(arr, (2,1,0)).astype(np.float32)
    else:
        raise RuntimeError("nibabel 또는 SimpleITK가 필요합니다.")

def robust_minmax_norm(vol, p1=1.0, p99=99.0, eps=1e-6):
    lo = np.percentile(vol, p1)
    hi = np.percentile(vol, p99)
    vol = np.clip(vol, lo, hi)
    return (vol - lo) / max(hi - lo, eps)

def otsu_threshold(arr, bins=256):
    arr = arr[np.isfinite(arr)]
    if arr.size == 0:
        return 0.0
    hist, bin_edges = np.histogram(arr, bins=bins, range=(arr.min(), arr.max()))
    w1 = np.cumsum(hist)
    w2 = np.cumsum(hist[::-1])[::-1]
    mu1 = np.cumsum(hist * (bin_edges[:-1] + bin_edges[1:]) / 2) / np.maximum(w1, 1e-12)
    mu2 = (np.cumsum((hist * (bin_edges[:-1] + bin_edges[1:]) / 2)[::-1]) / np.maximum(w2[::-1], 1e-12))[::-1]
    var12 = w1[:-1] * w2[1:] * (mu1[:-1] - mu2[1:])**2
    idx = np.argmax(var12)
    return float((bin_edges[idx] + bin_edges[idx+1]) / 2)

# ---- Metrics ----
def gradient_magnitude(s2d):
    if has_scipy:
        gx = sobel(s2d, axis=0, mode='reflect')
        gy = sobel(s2d, axis=1, mode='reflect')
    else:
        gx = np.zeros_like(s2d); gy = np.zeros_like(s2d)
        gx[1:-1,:] = (s2d[2:,:] - s2d[:-2,:]) / 2.0
        gy[:,1:-1] = (s2d[:,2:] - s2d[:,:-2]) / 2.0
    return np.sqrt(gx**2 + gy**2)

def laplacian2d(s2d):
    if has_scipy:
        return laplace(s2d)
    center = 4*s2d
    n = np.zeros_like(s2d)
    n[1: ,:] += s2d[:-1,:]; n[:-1,:] += s2d[1: ,:]
    n[: ,1:] += s2d[:, :-1]; n[: ,:-1] += s2d[:, 1: ]
    return n - center

def mse(a,b): return float(np.mean((a-b)**2))
def psnr_from_mse(m, data_range=1.0):
    if m <= 1e-12: return float('inf')
    return 20.0*math.log10(data_range) - 10.0*math.log10(m)

def global_ssim(a,b,data_range=1.0,K1=0.01,K2=0.03):
    a = a.astype(np.float64); b = b.astype(np.float64)
    C1 = (K1*data_range)**2; C2 = (K2*data_range)**2
    mu_a = float(np.mean(a)); mu_b = float(np.mean(b))
    var_a = float(np.var(a)); var_b = float(np.var(b))
    cov = float(np.mean((a - mu_a) * (b - mu_b)))
    num = (2*mu_a*mu_b + C1) * (2*cov + C2)
    den = (mu_a**2 + mu_b**2 + C1) * (var_a + var_b + C2)
    if den == 0: return 1.0 if num == 0 else 0.0
    return float(num/den)

def mutual_information(a,b,bins=64):
    a = np.clip(a.ravel(),0,1); b = np.clip(b.ravel(),0,1)
    h, _, _ = np.histogram2d(a,b,bins=bins,range=[[0,1],[0,1]])
    p = h/np.sum(h) + 1e-12
    px = np.sum(p,axis=1,keepdims=True); py = np.sum(p,axis=0,keepdims=True)
    return float(np.sum(p*(np.log(p) - np.log(px) - np.log(py))))

def ncc(a,b):
    a = a.astype(np.float64); b = b.astype(np.float64)
    am, bm = np.mean(a), np.mean(b)
    asd, bsd = np.std(a), np.std(b)
    if asd < 1e-8 or bsd < 1e-8: return 0.0
    return float(np.mean((a-am)*(b-bm)) / (asd*bsd))

def tv_z(vol, mask=None):
    diffs = np.abs(np.diff(vol, axis=2))
    if mask is not None:
        diffs = diffs * mask[:,:,1:]
        denom = np.sum(mask[:,:,1:])
        return float(np.sum(diffs) / max(denom, 1e-12))
    return float(np.mean(diffs))

def tv_z_per_interval(vol, mask=None):
    diffs = np.abs(np.diff(vol, axis=2))
    if mask is not None:
        diffs = diffs * mask[:,:,1:]
        denom = np.sum(mask[:,:,1:], axis=(0,1))
        denom = np.maximum(denom, 1e-12)
        return np.sum(diffs, axis=(0,1)) / denom
    return np.mean(diffs, axis=(0,1))

def tv_xy(vol, mask=None):
    dx = np.abs(np.diff(vol, axis=0))
    dy = np.abs(np.diff(vol, axis=1))
    if mask is not None:
        mx = mask[1:,:,:]; my = mask[:,1:,:]
        dx = dx * mx; dy = dy * my
        denom = np.sum(mx) + np.sum(my)
        return float((np.sum(dx)+np.sum(dy)) / max(denom, 1e-12))
    return float(np.mean(dx) + np.mean(dy))

def edge_tv_z(vol, mask=None):
    Z = vol.shape[2]
    gm = np.zeros_like(vol, dtype=np.float32)
    for z in range(Z):
        gm[:,:,z] = gradient_magnitude(vol[:,:,z])
    return tv_z(gm, mask)

def lap_tv_z(vol, mask=None):
    Z = vol.shape[2]
    lv = np.zeros_like(vol, dtype=np.float32)gradient_magnitude
    for z in range(Z):
        lv[:,:,z] = laplacian2d(vol[:,:,z])
    return tv_z(lv, mask)

def inplane_sharpness_metrics(vol, mask=None):
    Z = vol.shape[2]
    lap_abs, lap_var, edge_m = [], [], []
    for z in range(Z):
        sl = vol[:,:,z]
        if mask is not None:
            m = mask[:,:,z] > 0
            if not np.any(m):
                lap_abs.append(0.0); lap_var.append(0.0); edge_m.append(0.0); continue
            lap = laplacian2d(sl)[m]; gm = gradient_magnitude(sl)[m]
        else:
            lap = laplacian2d(sl).ravel(); gm = gradient_magnitude(sl).ravel()
        lap_abs.append(float(np.mean(np.abs(lap))))
        lap_var.append(float(np.var(lap)))
        edge_m.append(float(np.mean(gm)))
    return {
        "LaplacianAbs_mean": float(np.mean(lap_abs)),
        "LaplacianVar_mean": float(np.mean(lap_var)),
        "EdgeMag_mean": float(np.mean(edge_m)),
    }

def per_slice_pair_metrics(vol, mask=None):
    Z = vol.shape[2]
    d = {"MSE":[], "PSNR":[], "SSIM_global":[], "MI":[], "NCC":[], "TV":[]}
    intervals = tv_z_per_interval(vol, mask)
    for z in range(Z-1):
        a = vol[:,:,z]; b = vol[:,:,z+1]
        if mask is not None:
            m = (mask[:,:,z] * mask[:,:,z+1]) > 0
            a_sel = a[m] if np.any(m) else a
            b_sel = b[m] if np.any(m) else b
        else:
            a_sel = a; b_sel = b
        mv = mse(a_sel, b_sel)
        d["MSE"].append(mv)
        d["PSNR"].append(psnr_from_mse(mv))
        d["SSIM_global"].append(global_ssim(a_sel, b_sel))
        d["MI"].append(mutual_information(a_sel, b_sel))
        d["NCC"].append(ncc(a_sel, b_sel))
        d["TV"].append(float(intervals[z]))
    for k in d.keys():
        d[k] = np.array(d[k], dtype=np.float64)
    return d

def summarize_all(vol, name, spacing_z=1.0, use_mask=True):
    vol_n = robust_minmax_norm(vol)
    mask = None
    if use_mask:
        thr = otsu_threshold(vol_n)
        mask = (vol_n > thr).astype(np.float32)

    tvz = tv_z(vol_n, mask)
    tvxy = tv_xy(vol_n, mask)

    nz = vol_n[vol_n > 0]
    iqr = float(np.percentile(nz,75) - np.percentile(nz,25)) if nz.size > 0 else 1.0
    nTV_iqr = tvz / max(iqr, 1e-6)
    nTV_xy  = tvz / max(tvxy, 1e-6)

    edge_tv = edge_tv_z(vol_n, mask)
    lap_tv  = lap_tv_z(vol_n, mask)

    curves = per_slice_pair_metrics(vol_n, mask)
    sharp  = inplane_sharpness_metrics(vol_n, mask)

    return {
        "Name": name,
        "NumSlices": int(vol.shape[2]),
        "TV_z": tvz / max(spacing_z, 1e-6),  # spacing 보정
        "TV_xy": tvxy,
        "nTV_z_by_IQR": nTV_iqr,
        "nTV_z_by_TVxy": nTV_xy,
        "Edge_TV_z": edge_tv,
        "Lap_TV_z": lap_tv,
        "HF_LaplacianAbs_mean": sharp["LaplacianAbs_mean"],
        "HF_LaplacianVar_mean": sharp["LaplacianVar_mean"],
        "HF_EdgeMag_mean": sharp["EdgeMag_mean"],
        "PerSlice_Metrics": curves,
    }

# ---- Runner ----
def run(paths, base_out_dir, plots_dir, spacing_z=1.0, use_mask=True):
    os.makedirs(base_out_dir, exist_ok=True)
    os.makedirs(plots_dir, exist_ok=True)

    summaries = []
    per_slice_csvs = {}

    print(f"[INFO] Output dir: {base_out_dir}")
    for name, p in paths.items():
        if not os.path.exists(p):
            print(f"[WARN] Missing file: {p}")
            continue
        print(f"[LOAD] {name}: {p}")
        vol = load_nifti(p)
        if vol.ndim != 3:
            raise ValueError(f"{name}: expected 3D, got {vol.shape}")

        S = summarize_all(vol, name, spacing_z=spacing_z, use_mask=use_mask)
        summaries.append(S)

        # per-slice CSV (KEEP location/pattern)
        df = pd.DataFrame({k: v for k, v in S["PerSlice_Metrics"].items()})
        per_csv = os.path.join(base_out_dir, f"per_slice_{name}.csv")
        df.to_csv(per_csv, index=False)
        per_slice_csvs[name] = per_csv
        print(f"[SAVE] Per-slice metrics -> {per_csv}")

    if len(summaries) == 0:
        raise RuntimeError("No volumes processed. Check file paths.")

    # Summary CSV (KEEP filename)
    rows = []
    for S in summaries:
        rows.append({
            "Name": S["Name"],
            "NumSlices": S["NumSlices"],
            "TV_z": S["TV_z"],
            "TV_xy": S["TV_xy"],
            "nTV_z_by_IQR": S["nTV_z_by_IQR"],
            "nTV_z_by_TVxy": S["nTV_z_by_TVxy"],
            "Edge_TV_z": S["Edge_TV_z"],
            "Lap_TV_z": S["Lap_TV_z"],
            "HF_LaplacianAbs_mean": S["HF_LaplacianAbs_mean"],
            "HF_LaplacianVar_mean": S["HF_LaplacianVar_mean"],
            "HF_EdgeMag_mean": S["HF_EdgeMag_mean"],
            "PSNR_mean": float(np.mean(S["PerSlice_Metrics"]["PSNR"])),
            "SSIM_global_mean": float(np.mean(S["PerSlice_Metrics"]["SSIM_global"])),
            "MI_mean": float(np.mean(S["PerSlice_Metrics"]["MI"])),
            "NCC_mean": float(np.mean(S["PerSlice_Metrics"]["NCC"])),
            "TV_interval_mean": float(np.mean(S["PerSlice_Metrics"]["TV"])),
        })
    summary_df = pd.DataFrame(rows)
    summary_path = os.path.join(base_out_dir, "inter_slice_consistency_summary.csv")
    summary_df.to_csv(summary_path, index=False)
    print(f"[SAVE] Summary -> {summary_path}")

    # Plots (save into jbhi_metrics subfolder)
    def plot_metric(metric_key, ylabel):
        plt.figure()
        for S in summaries:
            y = S["PerSlice_Metrics"][metric_key]
            x = np.arange(1, len(y)+1)
            plt.plot(x, y, label=S["Name"])
        plt.xlabel("Adjacent slice index (z → z+1)")
        plt.ylabel(ylabel)
        plt.legend()
        out = os.path.join(plots_dir, f"plot_{metric_key}.png")
        plt.savefig(out, dpi=150, bbox_inches="tight")
        plt.close()
        print(f"[PLOT] {metric_key} -> {out}")

    for k, lab in [
        ("SSIM_global", "Global SSIM (z,z+1)"),
        ("PSNR", "PSNR (dB) (z,z+1)"),
        ("MI", "Mutual Information (z,z+1)"),
        ("NCC", "Normalized Cross-Correlation (z,z+1)"),
        ("TV", "Total Variation per z-interval"),
    ]:
        try:
            plot_metric(k, lab)
        except Exception as e:
            print(f"[WARN] Plot fail ({k}): {e}")

    print("\n=== Summary (lower is better for TV metrics; higher is better for PSNR/SSIM/MI/NCC) ===")
    print(summary_df.to_string(index=False))

    return summary_path, per_slice_csvs

# ---- Execute ----
if __name__ == "__main__":
    run(paths, base_out_dir, plots_dir, spacing_z=1.0, use_mask=True)


[INFO] Output dir: /SSD5_8TB/Daniel/JBHI 저널/Slice간의 일관성 정량값
[LOAD] ProposedSynth_Epoch87_fullimage: /SSD5_8TB/Daniel/JBHI 저널/Output(result)/MR-CT/Registformer_MrCtPelvis_ver2_ProposedSynthesis_NoRandTranslation_VoxelmorphProposedSynth_Test_Epoch87_fullimage/preds_a_1PC066.nii.gz
[SAVE] Per-slice metrics -> /SSD5_8TB/Daniel/JBHI 저널/Slice간의 일관성 정량값/per_slice_ProposedSynth_Epoch87_fullimage.csv
[LOAD] MUNIT_Lcx_rotate: /SSD5_8TB/Daniel/JBHI 저널/Output(result)/MR-CT/I2I_translation/MUNIT+Lcx(예전)_rotate/preds_b_1PC066.nii.gz
[SAVE] Per-slice metrics -> /SSD5_8TB/Daniel/JBHI 저널/Slice간의 일관성 정량값/per_slice_MUNIT_Lcx_rotate.csv
[LOAD] VoxelMorph2D_warped: /SSD5_8TB/Daniel/JBHI 저널/Output(result)/MR-CT/Registration/voxelmorph2D/warped_img_1PC066.nii.gz
[SAVE] Per-slice metrics -> /SSD5_8TB/Daniel/JBHI 저널/Slice간의 일관성 정량값/per_slice_VoxelMorph2D_warped.csv
[SAVE] Summary -> /SSD5_8TB/Daniel/JBHI 저널/Slice간의 일관성 정량값/inter_slice_consistency_summary.csv
[PLOT] SSIM_global -> /SSD5_8TB/Daniel/JBHI 저널/Slice